In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Sep 28 11:44:24 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [4]:
import json
import ast
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
import sklearn.preprocessing as preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import ClassifierChain
from sklearn.svm import SVC


import random

from sklearn.naive_bayes import GaussianNB,MultinomialNB
from sklearn.metrics import accuracy_score,hamming_loss
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score



In [5]:
train_file_path = "/content/drive/MyDrive/assign2/comp90051-22-s2-authorship/train.json"
test_file_path = "/content/drive/MyDrive/assign2/comp90051-22-s2-authorship/test.json"

with open(train_file_path, 'r') as f:
    data = json.load(f)
df_train = pd.DataFrame(data)

with open(test_file_path, 'r') as f:
    data = json.load(f)
df_test = pd.DataFrame(data)

In [6]:
# Retrive the non-prolific author and prolific author index
non_prolific_author_index = []
prolific_author_index = []
for i,j in enumerate(df_train['authors']):
  sum = 0
  for k in j:
    if k < 100 and k >= 0:
      sum += 1
  if sum == 0:
    non_prolific_author_index.append(i)
  else:
    prolific_author_index.append(i)

df_non_prolifc_authors = df_train.drop(prolific_author_index)
df_prolific_authors = df_train.drop(non_prolific_author_index)

In [7]:
df_prolific_authors

,authors,year,abstract,venue,title
0,"[42, 13720, 36]",9,"[2455, 1858, 2335, 1543, 1800, 1860, 2000, 286...",20,"[41, 1550, 1563, 1594, 1544, 1919, 1644, 37, 1..."
1,"[1359, 15881, 45]",15,"[40, 1542, 1691, 2449, 1535, 3616, 2206, 1904,...",2,"[1731, 47, 11, 57, 4624, 1525, 1535, 47, 11, 3..."
3,[97],10,"[46, 1624, 1547, 56, 1687, 1644, 6, 7, 3386, 1...",4,"[40, 1733, 1735, 1540, 1655, 46, 1624, 1547, 5..."
4,"[19617, 2]",10,"[37, 3709, 3836, 1586, 2151, 1727, 3021, 1860,...",9,"[38, 1592, 2088, 1543, 1574, 1727, 1597, 1813,..."
9,"[9641, 44, 5623, 2]",18,"[1731, 2021, 1543, 11, 1546, 11, 1647, 2163, 1...",0,"[1560, 1694, 11, 1546, 11, 3066, 1728, 47, 160..."
...,...,...,...,...,...
25767,"[19974, 68, 8903]",1,"[40, 1542, 1691, 2449, 1535, 2855, 1549, 1755,...",23,"[1708, 33, 58, 57, 2174, 1548, 1538, 1553, 359..."
25776,"[6353, 67, 3037, 15856, 13521]",10,"[40, 1542, 1691, 2907, 1540, 1535, 2121, 2431,...",10,"[53, 2555, 4480, 1535, 1637, 1898, 1549, 53, 3..."
25778,"[7686, 3810, 87, 7451]",16,"[2144, 1661, 2151, 1966, 3776, 4864, 1546, 219...",31,"[1615, 1966, 11, 3238, 1653, 3318, 24, 2144, 1..."
25781,"[14864, 92]",16,"[37, 1662, 33, 1814, 2036, 1692, 1553, 2333, 2...",0,"[46, 1670, 1826, 37, 1632, 46, 1991, 1525, 152..."


In [8]:
df_non_prolifc_authors

,authors,year,abstract,venue,title
2,"[19166, 17763]",17,"[40, 1542, 1691, 2449, 1535, 2610, 1543, 1535,...",,"[2085, 1719, 1846, 1745, 2243, 1553, 1606, 159..."
5,"[5396, 16841, 17598, 20375, 11436]",18,"[40, 1542, 1691, 3133, 33, 1923, 1650, 1642, 1...",4,"[46, 1617, 1667, 3979, 2073, 37, 53, 2080, 154..."
6,"[4874, 20811]",14,"[37, 2067, 1681, 1866, 51, 1612, 43, 1545, 181...",0,"[34, 3646, 2073, 2035, 2346, 1886, 1543, 57, 1..."
7,"[1398, 11095]",10,"[1664, 1575, 3781, 4692, 10, 1706, 1860, 2000,...",26,"[1615, 1966, 11, 3495, 1656, 4345, 24, 2353, 1..."
8,"[12449, 20348]",18,"[46, 4822, 1692, 1553, 3591, 4914, 46, 2421, 6...",9,"[3591, 4914, 46, 2421, 1608, 37, 1740, 1825, 1..."
...,...,...,...,...,...
25787,"[11958, 20085]",14,"[44, 4221, 1535, 1848, 1543, 1839, 1866, 2828,...",20,"[3207, 3592, 1535, 2499, 1543, 4199, 1546, 237..."
25789,"[18357, 11563, 7905]",11,"[46, 1605, 1691, 10, 2551, 4168, 2030, 3134, 1...",278,"[47, 1574, 1541, 1549, 47, 1966, 2114, 112, 12..."
25790,"[2236, 20436, 10501]",7,"[1611, 1543, 1535, 2817, 2073, 1647, 11, 2933,...",17,"[56, 1718, 3012, 56, 1902, 2160, 1546, 1623, 1..."
25791,"[3921, 8760]",16,"[37, 1662, 33, 2007, 1669, 4981, 1650, 1527, 1...",0,"[3066, 1728, 11, 1546, 11, 3066, 1728, 1531, 1..."


In [9]:
selected_col_nums = round(200*12.5)
non_prolific_author_rows =[]

for i in range(selected_col_nums):
  non_prolific_author_rows.append(random.randint(0,18333))

df_selected_non_prolific_authors = df_non_prolifc_authors.iloc[non_prolific_author_rows]



In [10]:
df_selected_non_prolific_authors

,authors,year,abstract,venue,title
18938,"[19246, 19098, 4620, 8558, 19814]",12,"[47, 2376, 51, 2324, 56, 1687, 1644, 1654, 37,...",343,"[56, 1687, 1644, 1556, 2068, 114, 38, 114, 164..."
25055,"[2612, 15998, 635, 10729]",4,"[37, 2227, 33, 2044, 1701, 1553, 2410, 2890, 1...",0,"[36, 1664, 4480, 3735, 1575, 1549, 1542, 3034,..."
7691,"[12269, 15841, 7351]",13,"[1533, 2231, 2399, 1843, 1554, 46, 1624, 1547,...",42,"[46, 3827, 1538, 1542, 2048, 1852, 2810, 3111,..."
7921,"[12812, 2179]",8,"[1530, 2104, 1905, 3455, 4082, 1535, 2500, 213...",4,"[1751, 44, 1619, 1872, 24, 47, 2376, 51, 33, 1..."
11797,"[10491, 20153, 9559]",16,"[37, 4021, 1525, 1535, 2223, 1656, 1527, 2359,...",31,"[1731, 37, 1529, 1538, 1803, 2322, 1539, 1615,..."
...,...,...,...,...,...
23406,"[20142, 10167, 12944, 777]",5,"[46, 1605, 1691, 10, 1557, 1662, 33, 3247, 40,...",42,"[57, 2823, 40, 1621, 1536, 1529, 1589, 1629, 4..."
5035,"[17725, 17510]",15,"[1697, 1545, 2137, 2259, 1542, 33, 3128, 1656,...",13,"[1697, 1545, 46, 1528, 1727, 37, 3878, 1537, 1..."
2049,"[19844, 4611, 9136, 7341]",13,"[40, 1542, 1691, 3133, 33, 2095, 1701, 1553, 1...",9,"[1527, 1611, 2208, 1529, 1589, 1629, 1549, 183..."
2326,"[16012, 15763]",18,"[37, 2067, 4952, 1525, 1539, 1740, 1529, 1589,...",4,"[1531, 1530, 36, 1664, 1820, 1549, 1535, 47, 1..."


In [11]:
# concate the selected non-prolific authors with the prolific authors
df_prolific_authors = pd.concat([df_selected_non_prolific_authors, df_prolific_authors], axis = 0)

In [12]:
df_prolific_authors

,authors,year,abstract,venue,title
18938,"[19246, 19098, 4620, 8558, 19814]",12,"[47, 2376, 51, 2324, 56, 1687, 1644, 1654, 37,...",343,"[56, 1687, 1644, 1556, 2068, 114, 38, 114, 164..."
25055,"[2612, 15998, 635, 10729]",4,"[37, 2227, 33, 2044, 1701, 1553, 2410, 2890, 1...",0,"[36, 1664, 4480, 3735, 1575, 1549, 1542, 3034,..."
7691,"[12269, 15841, 7351]",13,"[1533, 2231, 2399, 1843, 1554, 46, 1624, 1547,...",42,"[46, 3827, 1538, 1542, 2048, 1852, 2810, 3111,..."
7921,"[12812, 2179]",8,"[1530, 2104, 1905, 3455, 4082, 1535, 2500, 213...",4,"[1751, 44, 1619, 1872, 24, 47, 2376, 51, 33, 1..."
11797,"[10491, 20153, 9559]",16,"[37, 4021, 1525, 1535, 2223, 1656, 1527, 2359,...",31,"[1731, 37, 1529, 1538, 1803, 2322, 1539, 1615,..."
...,...,...,...,...,...
25767,"[19974, 68, 8903]",1,"[40, 1542, 1691, 2449, 1535, 2855, 1549, 1755,...",23,"[1708, 33, 58, 57, 2174, 1548, 1538, 1553, 359..."
25776,"[6353, 67, 3037, 15856, 13521]",10,"[40, 1542, 1691, 2907, 1540, 1535, 2121, 2431,...",10,"[53, 2555, 4480, 1535, 1637, 1898, 1549, 53, 3..."
25778,"[7686, 3810, 87, 7451]",16,"[2144, 1661, 2151, 1966, 3776, 4864, 1546, 219...",31,"[1615, 1966, 11, 3238, 1653, 3318, 24, 2144, 1..."
25781,"[14864, 92]",16,"[37, 1662, 33, 1814, 2036, 1692, 1553, 2333, 2...",0,"[46, 1670, 1826, 37, 1632, 46, 1991, 1525, 152..."


In [13]:
# Retrieve the prolific author as label
prolific_author_label = []
non_prolific_author_label = []
for i in df_prolific_authors['authors']:
  prolific_author_label_each_instance = []
  non_prolific_author_label_each_instance = []
  for j in i:
    if j >= 0 and j < 100:
      prolific_author_label_each_instance.append(j)
    else:
      non_prolific_author_label_each_instance.append(j)
  if len(prolific_author_label_each_instance) == 0:
     prolific_author_label_each_instance.append(100)

  prolific_author_label.append(prolific_author_label_each_instance)
  non_prolific_author_label.append(non_prolific_author_label_each_instance)


df_prolific_authors['coauthors'] = non_prolific_author_label

In [14]:
#One-hot encoding the labels
multilable_binarizer = MultiLabelBinarizer()
ohe_prolific_labels = multilable_binarizer.fit_transform(prolific_author_label).astype('float')

In [15]:
ohe_prolific_labels.shape

(9960, 101)

In [16]:
df_prolific_authors['sentences'] = df_prolific_authors['title'] + df_prolific_authors['abstract'] 
df_prolific_authors = df_prolific_authors.drop(['abstract','title'],axis = 1)

df_test['sentences'] = df_test['title'] + df_test['abstract'] 
df_test = df_test.drop(['abstract','title'],axis = 1)

In [17]:
df_prolific_authors

,authors,year,venue,coauthors,sentences
18938,"[19246, 19098, 4620, 8558, 19814]",12,343,"[19246, 19098, 4620, 8558, 19814]","[56, 1687, 1644, 1556, 2068, 114, 38, 114, 164..."
25055,"[2612, 15998, 635, 10729]",4,0,"[2612, 15998, 635, 10729]","[36, 1664, 4480, 3735, 1575, 1549, 1542, 3034,..."
7691,"[12269, 15841, 7351]",13,42,"[12269, 15841, 7351]","[46, 3827, 1538, 1542, 2048, 1852, 2810, 3111,..."
7921,"[12812, 2179]",8,4,"[12812, 2179]","[1751, 44, 1619, 1872, 24, 47, 2376, 51, 33, 1..."
11797,"[10491, 20153, 9559]",16,31,"[10491, 20153, 9559]","[1731, 37, 1529, 1538, 1803, 2322, 1539, 1615,..."
...,...,...,...,...,...
25767,"[19974, 68, 8903]",1,23,"[19974, 8903]","[1708, 33, 58, 57, 2174, 1548, 1538, 1553, 359..."
25776,"[6353, 67, 3037, 15856, 13521]",10,10,"[6353, 3037, 15856, 13521]","[53, 2555, 4480, 1535, 1637, 1898, 1549, 53, 3..."
25778,"[7686, 3810, 87, 7451]",16,31,"[7686, 3810, 7451]","[1615, 1966, 11, 3238, 1653, 3318, 24, 2144, 1..."
25781,"[14864, 92]",16,0,[14864],"[46, 1670, 1826, 37, 1632, 46, 1991, 1525, 152..."


In [18]:
# coauthors_ohe = multilable_binarizer.fit_transform(df_prolific_authors['coauthors']).astype('float')
# sentences_ohe = multilable_binarizer.fit_transform(df_prolific_authors['sentences']).astype('float')
# df_prolific_authors['coauthors'])

def convert_to_string(df):
  total_list = []
  for i in df:
    total_list.append(str(i)[1:-1].replace(",", ""))
  return total_list




# Counts for sentences
sentences_voc = {}
sentences_num = 4999
for i in range(sentences_num):
  sentences_voc[str(i)] = i

string_sentences = convert_to_string(df_prolific_authors['sentences'])
sentences_vectorizer = CountVectorizer(vocabulary = sentences_voc)
sentences_counts = sentences_vectorizer.fit_transform(string_sentences).toarray()  

# Counts for coauthors
coauthor_voc = {}
coauthor_num = 21246
for i in range(coauthor_num):
  coauthor_voc[str(i)] = i

string_coauthor = convert_to_string(df_prolific_authors['coauthors'])
coauthor_vectorizer = CountVectorizer(vocabulary = coauthor_voc)
coauthor_counts = coauthor_vectorizer.fit_transform(string_coauthor).toarray()





In [19]:

coauthors_features_name = ['coauthors_' + str(i) for i in range(21246)]
# coauthors_features = np.vstack([coauthors_features_name,coauthors_ohe])
df_coauthors_features = pd.DataFrame(coauthor_counts)
df_coauthors_features = df_coauthors_features.astype('float')
df_coauthors_features.columns = coauthors_features_name

sentence_features_name = ['sentences_' + str(i) for i in range(1,5000)]
df_sentence_features = pd.DataFrame(sentences_counts)
df_sentence_features = df_sentence_features.astype('float')
df_sentence_features.columns = sentence_features_name


df_merge_coauthors_sentences = pd.concat([df_coauthors_features, df_sentence_features], axis = 1)

In [20]:
df_merge_coauthors_sentences['coauthors_13720']

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
9955    0.0
9956    0.0
9957    0.0
9958    0.0
9959    0.0
Name: coauthors_13720, Length: 9960, dtype: float64

In [21]:
X_train,X_test,y_train,y_test = train_test_split(df_merge_coauthors_sentences,ohe_prolific_labels,test_size=0.1,random_state=90051)

In [ ]:
base_lr = LogisticRegression(solver='lbfgs', random_state=0,max_iter=150,n_jobs = -1)
chain = ClassifierChain(base_lr, order='random', random_state=0)
scaler = preprocessing.StandardScaler().fit(X_train)
X_scaled_train = scaler.transform(X_train)
X_scaled_test= scaler.transform(X_test)
chain.fit(X_scaled_train, y_train)
predictions = chain.predict(X_scaled_test)
f1_score(y_test, predictions, average=None)


In [ ]:
f1_score(y_test, predictions, average='samples')

In [ ]:
#Test
# Counts for sentences
sentences_voc = {}
sentences_num = 4999
for i in range(sentences_num):
  sentences_voc[str(i)] = i

test_string_sentences = convert_to_string(df_test['sentences'])
test_sentences_vectorizer = CountVectorizer(vocabulary = sentences_voc)
test_sentences_counts = test_sentences_vectorizer.fit_transform(test_string_sentences).toarray()  

# Counts for coauthors
coauthor_voc = {}
coauthor_num = 21246
for i in range(coauthor_num):
  coauthor_voc[str(i)] = i

test_string_coauthor = convert_to_string(df_test['coauthors'])
test_coauthor_vectorizer = CountVectorizer(vocabulary = coauthor_voc)
test_coauthor_counts = coauthor_vectorizer.fit_transform(test_string_coauthor).toarray()



test_coauthors_features_name = ['coauthors_' + str(i) for i in range(21246)]
# coauthors_features = np.vstack([coauthors_features_name,coauthors_ohe])
df_test_coauthors_features = pd.DataFrame(test_coauthor_counts)
df_test_coauthors_features = df_test_coauthors_features.astype('float')
df_test_coauthors_features.columns = test_coauthors_features_name

test_sentence_features_name = ['sentences_' + str(i) for i in range(1,5000)]
df_test_sentence_features = pd.DataFrame(test_sentences_counts)
df_test_sentence_features = df_test_sentence_features.astype('float')
df_test_sentence_features.columns = test_sentence_features_name


df_test_merge_coauthors_sentences = pd.concat([df_test_coauthors_features, df_test_sentence_features], axis = 1)




In [ ]:
df_test_merge_coauthors_sentences

In [ ]:
X_scaled_predict= scaler.transform(df_test_merge_coauthors_sentences)

In [ ]:
predition = chain.predict(X_scaled_predict)

In [ ]:
predition[3][100]

In [ ]:
predicted_list = [i.tolist() for i in predition]
total_index_list = []
for i in predicted_list:
  index_list = []
  for j,k in enumerate(i):
    if k == 1:
      if j == 100:
        index_list.append(str(-1))
      else:
        index_list.append(str(j))
  if len(index_list) != 0:
    total_index_list.append(index_list)
  else:
    total_index_list.append([str(-1)])

predicted_str = []
for i in total_index_list:
  a = ','.join(i)
  predicted_str.append(a)

In [ ]:

df_output = pd.DataFrame({'ID': list(range(800)),'Predict':predicted_str})
df_output.to_csv('predicted_result.csv',index = False)